In [1]:
import pandas as pd
import os
from tqdm import tqdm
import torchvision.models as models
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn

CSV_PATH = "E:\\study\\bachelor\\temp-files\\data\\Obtain data\\dataframes\\3_min_training_map.csv"

df = pd.read_csv(CSV_PATH)

for index, row in df.iterrows():
    if not os.path.exists(row['image_path']):
        df.drop(index, inplace=True)

In [2]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):

        img_path = self.dataframe.iloc[idx]['image_path']
        speed = self.dataframe.iloc[idx]['speed:']
        cur_gear = self.dataframe.iloc[idx]['curGear:']
        side_speed = self.dataframe.iloc[idx]['sideSpeed:']
        input_value = self.dataframe.iloc[idx]['inputSteer:']
        
        image = Image.open(img_path)
        
        if self.transform:
            image = self.transform(image)

        input_value = round(input_value, 1)
        
        return image, speed, cur_gear, side_speed, input_value

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

In [3]:
from fastbook import *

In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)

train_dataset = CustomDataset(dataframe=train_df, transform=transform)
test_dataset = CustomDataset(dataframe=test_df, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


In [5]:
learning_rate = 0.001
num_epochs = 100


class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        vgg16 = models.vgg16(pretrained=True)

        self.vgg16 = nn.Sequential(*list(vgg16.children())[:-1])
        
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 512),  # Adjust input and output sizes as needed
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1)  # Output a single continuous value
        )

    def forward(self, image):
        extracted_features = self.vgg16(image)

        predicted_input = self.head(extracted_features)

        return predicted_input


model = SimpleModel()

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

e:\study\bachelor\github\trackmania-ai\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\study\bachelor\github\trackmania-ai\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SimpleModel(
  (vgg16): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (1

In [6]:
for epoch in tqdm(range(num_epochs)):
    for batch in train_loader:
        image, speed, cur_gear, side_speed, input_value = batch
        image = image.to(torch.float32)
        input_value = input_value.to(torch.float32)
        image, speed, cur_gear, side_speed, input_value = image.to(device), speed.to(device), cur_gear.to(device), side_speed.to(device), input_value.to(device)

        optimizer.zero_grad()  # Обнулення градієнтів
        outputs = model(image)  # Пропускаємо дані через модель
        # print((outputs, input_value))
        loss = criterion(outputs, input_value)  # Обчислюємо втрати
        loss.backward()  # Зворотнє поширення помилки
        optimizer.step()  # Оновлення ваг моделі
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

  0%|          | 0/100 [00:00<?, ?it/s]

e:\study\bachelor\github\trackmania-ai\.venv\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
e:\study\bachelor\github\trackmania-ai\.venv\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 1/100 [03:50<6:19:36, 230.06s/it]

Epoch [1/100], Loss: 0.6188


  1%|          | 1/100 [06:43<11:06:32, 403.96s/it]


KeyboardInterrupt: 